# Supervised Learning: Scratch vs scikit-learn

Examples showing the math behind basic algorithms and their scikit-learn counterparts.

In [ ]:
!pip install scikit-learn matplotlib -q

In [ ]:
import numpy as npfrom sklearn.datasets import make_regression, make_classification, load_digits, load_irisfrom sklearn.model_selection import train_test_splitfrom sklearn.linear_model import LinearRegression, Perceptron, LogisticRegressionfrom sklearn.neighbors import KNeighborsClassifierfrom sklearn.preprocessing import PolynomialFeaturesfrom sklearn.pipeline import make_pipelinefrom sklearn.metrics import accuracy_score, ConfusionMatrixDisplayimport matplotlib.pyplot as plt

## 0. Linear regression from scratch

In [ ]:
X, y, true_coef = make_regression(n_samples=100, n_features=1, noise=10.0, coef=True, random_state=42)X_b = np.c_[np.ones((len(X),1)), X]theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ yprint("True coef:", true_coef)print("Closed-form coef:", theta[1])print("Intercept:", theta[0])

In [ ]:
x_grid = np.linspace(X.min(), X.max(), 100).reshape(-1,1)y_pred = theta[1]*x_grid + theta[0]plt.scatter(X, y, color="blue", label="data")plt.plot(x_grid, y_pred, color="red", label="fit")plt.legend(); plt.tight_layout(); plt.show()

### scikit-learn linear regression

In [ ]:
model = LinearRegression()model.fit(X, y)print("sklearn coef:", model.coef_[0])print("sklearn intercept:", model.intercept_)

## 1. k-NN classification from scratch

In [ ]:
X, y = load_digits(return_X_y=True)X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)def knn_predict(X_train, y_train, X_test, k=3):    preds = []    for x in X_test:        dists = np.linalg.norm(X_train - x, axis=1)        idx = np.argsort(dists)[:k]        preds.append(np.bincount(y_train[idx]).argmax())    return np.array(preds)preds = knn_predict(X_train, y_train, X_test, k=3)print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn k-NN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)knn.fit(X_train, y_train)sk_preds = knn.predict(X_test)print("Accuracy:", accuracy_score(y_test, sk_preds))

## 2. Perceptron with gradient descent

In [ ]:
X, y = make_classification(n_samples=200, n_features=2, n_informative=2, n_redundant=0, random_state=42)X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)y_train_signed = np.where(y_train==0, -1, 1)w = np.zeros(X_train.shape[1])b = 0.for _ in range(100):    margins = y_train_signed * (X_train @ w + b)    mask = margins < 0    if not mask.any():        break    grad_w = -(y_train_signed[mask,None] * X_train[mask]).mean(axis=0)    grad_b = -(y_train_signed[mask]).mean()    w -= 0.1 * grad_w    b -= 0.1 * grad_bpreds = (X_test @ w + b >= 0).astype(int)print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn Perceptron

In [ ]:
skp = Perceptron(max_iter=1000, eta0=0.1, tol=1e-3)skp.fit(X_train, y_train)sk_preds = skp.predict(X_test)print("Accuracy:", accuracy_score(y_test, sk_preds))

## 3. Logistic regression from scratch

In [ ]:
X, y = load_iris(return_X_y=True)mask = y < 2X = X[mask, :2]y = y[mask]X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)w = np.zeros(X_train.shape[1])b = 0.def sigmoid(z):    return 1/(1+np.exp(-z))for _ in range(200):    z = X_train @ w + b    preds = sigmoid(z)    grad_w = X_train.T @ (preds - y_train) / len(y_train)    grad_b = np.mean(preds - y_train)    w -= 0.1 * grad_w    b -= 0.1 * grad_bpreds = (sigmoid(X_test @ w + b) >= 0.5).astype(int)print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn logistic regression

In [ ]:
clf = LogisticRegression(max_iter=200)clf.fit(X_train, y_train)sk_preds = clf.predict(X_test)print("Accuracy:", accuracy_score(y_test, sk_preds))

This concludes the basic examples.